# near_workspaces

In [ ]:
open rust
open rust.rust_operators

In [ ]:
//// test

open testing

## near

### near_token_workspaces

In [ ]:
nominal near_token_workspaces =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"near_workspaces::types::NearToken\")>]\n#endif\ntype near_workspaces_types_NearToken = class end"
        $'' : $'near_workspaces_types_NearToken'
    )

### gas

In [ ]:
nominal gas =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"near_workspaces::types::Gas\")>]\n#endif\ntype near_workspaces_types_Gas = class end"
        $'' : $'near_workspaces_types_Gas'
    )

### near_workspaces_error

In [ ]:
nominal near_workspaces_error =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"near_workspaces::error::Error\")>]\n#endif\ntype near_workspaces_error_Error = class end"
        $'' : $'near_workspaces_error_Error'
    )

### sandbox

In [ ]:
nominal sandbox =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"near_workspaces::network::Sandbox\")>]\n#endif\ntype near_workspaces_network_Sandbox = class end"
        $'' : $'near_workspaces_network_Sandbox'
    )

### worker

In [ ]:
nominal worker t =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"near_workspaces::Worker<$0>\")>]\n#endif\ntype near_workspaces_Worker<'T> = class end"
        $'' : $'near_workspaces_Worker<`t>'
    )

### contract

In [ ]:
nominal contract =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"near_workspaces::Contract\")>]\n#endif\ntype near_workspaces_Contract = class end"
        $'' : $'near_workspaces_Contract'
    )

### call_transaction

In [ ]:
nominal call_transaction =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"near_workspaces::operations::CallTransaction\")>]\n#endif\ntype near_workspaces_operations_CallTransaction = class end"
        $'' : $'near_workspaces_operations_CallTransaction'
    )

### execution_final_result

In [ ]:
nominal execution_final_result =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"near_workspaces::result::ExecutionFinalResult\")>]\n#endif\ntype near_workspaces_result_ExecutionFinalResult = class end"
        $'' : $'near_workspaces_result_ExecutionFinalResult'
    )

### execution_result

In [ ]:
nominal execution_result t =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"near_workspaces::result::ExecutionResult<$0>\")>]\n#endif\ntype near_workspaces_result_ExecutionResult<'T> = class end"
        $'' : $'near_workspaces_result_ExecutionResult<`t>'
    )

### execution_success

In [ ]:
nominal execution_success =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"near_workspaces::result::ExecutionSuccess\")>]\n#endif\ntype near_workspaces_result_ExecutionSuccess = class end"
        $'' : $'near_workspaces_result_ExecutionSuccess'
    )

### execution_failure

In [ ]:
nominal execution_failure =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"near_workspaces::result::ExecutionFailure\")>]\n#endif\ntype near_workspaces_result_ExecutionFailure = class end"
        $'' : $'near_workspaces_result_ExecutionFailure'
    )

### execution_outcome

In [ ]:
nominal execution_outcome =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"near_workspaces::result::ExecutionOutcome\")>]\n#endif\ntype near_workspaces_result_ExecutionOutcome = class end"
        $'' : $'near_workspaces_result_ExecutionOutcome'
    )

### sandbox_worker

In [ ]:
inl sandbox_worker () : resultm.result' (worker sandbox) near_workspaces_error =
    !\($'"near_workspaces::sandbox().await"')

### dev_deploy

In [ ]:
inl dev_deploy
    (wasm : am'.vec u8)
    (worker : worker sandbox)
    : async.future_pin (resultm.result' contract near_workspaces_error)
    =
    inl worker = worker |> rust.emit
    !\\(wasm, $'"Box::pin(!worker.dev_deploy(&$0))"')

### call

In [ ]:
inl call (fn_name : string) (contract : contract) : call_transaction =
    !\\((contract, fn_name), $'"$0.call(&*$1)"')

### logs

In [ ]:
inl logs (result : execution_final_result) : am'.vec (rust.ref sm'.str) =
    !\($'"!result.logs()"')

### into_result

In [ ]:
inl into_result
    (result : execution_final_result)
    : resultm.result' execution_success execution_failure
    =
    !\\(result, $'"$0.into_result()"')

### receipt_failures

In [ ]:
inl receipt_failures (result : execution_final_result) : am'.vec (rust.ref execution_outcome) =
    inl result = join result
    !\($'"!result.receipt_failures()"')

### receipt_outcomes

In [ ]:
inl receipt_outcomes (result : execution_final_result) : am'.vec execution_outcome =
    inl result = join result
    inl result : rust.ref (am'.slice execution_outcome) = !\($'"!result.receipt_outcomes()"')
    result |> rust.into

### json

In [ ]:
inl json (result : execution_final_result) : resultm.result' sm'.std_string near_workspaces_error =
    !\\(result, $'"$0.json()"')

### borsh

In [ ]:
inl borsh (result : execution_final_result) : resultm.result' sm'.std_string near_workspaces_error =
    !\\(result, $'"$0.borsh()"')

### total_gas_burnt

In [ ]:
inl total_gas_burnt (result : execution_final_result) : gas =
    !\\(result, $'"$0.total_gas_burnt"')

### as_gas

In [ ]:
inl as_gas (gas : gas) : u64 =
    !\\(gas, $'"$0.as_gas()"')

### outcomes

In [ ]:
inl outcomes (result : execution_final_result) : am'.vec (rust.ref execution_outcome) =
    inl result = result |> rust.emit
    !\($'"!result.outcomes()"')

### is_success

In [ ]:
inl is_success (outcome : execution_outcome) : bool =
    !\\(outcome, $'"$0.is_success()"')

### gas_burnt

In [ ]:
inl gas_burnt (outcome : execution_outcome) : gas =
    !\\(outcome, $'"$0.gas_burnt"')

### tokens_burnt

In [ ]:
inl tokens_burnt (outcome : execution_outcome) : near_token_workspaces =
    !\\(outcome, $'"$0.tokens_burnt"')

### transact

In [ ]:
inl transact
    (call : call_transaction)
    : async.future_pin (resultm.result' execution_final_result near_workspaces_error)
    =
    !\($'"Box::pin(!call.transact())"')

### gas

In [ ]:
inl gas
    (gas : gas)
    (call : call_transaction)
    : call_transaction
    =
    !\($'"!call.gas(!gas)"')

### from_tgas

In [ ]:
inl from_tgas
    (tgas : i32)
    : gas
    =
    !\($'"near_workspaces::types::Gas::from_tgas(!tgas)"')

### print_usd

In [ ]:
inl print_usd retry (result : execution_final_result) =
    inl total_gas_burnt = result |> total_gas_burnt |> as_gas
    inl total_gas_burnt_usd = total_gas_burnt |> near.gas_to_usd

    trace Info
        fun () => "near_workspaces.print_usd"
        fun () => { retry total_gas_burnt_usd total_gas_burnt }

    result
    |> outcomes
    |> iter.into_iter
    |> iter.cloned
    |> iter.for_each fun outcome =>
        inl is_success = outcome |> is_success

        inl gas_burnt = outcome |> gas_burnt |> as_gas
        inl gas_burnt_usd = gas_burnt |> near.gas_to_usd

        inl tokens_burnt = outcome |> tokens_burnt |> near.as_yoctonear
        inl tokens_burnt_usd = tokens_burnt |> near.tokens_to_usd

        trace Info
            fun () => "near_workspaces.print_usd / outcome"
            fun () => { is_success gas_burnt_usd tokens_burnt_usd gas_burnt tokens_burnt }